Run this script to combine Land Use (2020), Zoning, and Building Permit Data to identify progress and remaining development potential in developing subdivisions

Data sources:

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Land_Use_2020/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Building_Permit_Records/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Planimetrics/FeatureServer

https://gis2.lawrenceks.org/ArcGIS/rest/services/Parcels/MapServer

https://tiles.arcgis.com/tiles/8O9UlSTnqjKptoda/arcgis/rest/services/LAWRENCE_KS_AERIAL_2020/MapServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Lawrence_Zoning/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Plats/FeatureServer



In [ ]:
filegdb = r"C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210401.gdb"
arcpy.conversion.FeatureClassToFeatureClass("Plats", filegdb, "PlatsCopy", "#", '#', '')
arcpy.management.AddField(filegdb+r"\PlatsCopy", "LotBlock", "TEXT", None, None, 24, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(filegdb+r"\PlatsCopy", "LotBlock", 'str(!BLOCK!)+"-"+ str(!LOT!)', "PYTHON3", '', "TEXT")
arcpy.management.FeatureToPoint("PlatsCopy", filegdb+r"\PlatsCopyPoints", "INSIDE")
arcpy.analysis.SpatialJoin("Parcels", "PlatsCopyPoints", filegdb+r"\PlatsInParcels", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", None, '')
arcpy.management.AlterField("PlatsInParcels", "Join_Count", "Lots_in_Parcel", "Platted Lots Per Parcel", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.FeatureToPoint("PlatsInParcels", filegdb+r"\PlatsInParcelPoint", "INSIDE")
arcpy.analysis.SpatialJoin(filegdb+r"\PlatsInParcelPoint", "Lawrence Zoning", filegdb+r"\ParcelZoning", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField("ParcelZoning", "Join_Count", "Parcel_zone_Count", "Zonings Per Parcel", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.AddIndex(filegdb+r"\PlatsInParcels", "JOINPIN", "JOINPIN", "UNIQUE", "ASCENDING")
arcpy.management.AddIndex(filegdb+r"\ParcelZoning", "JOINPIN", "JOINPINz", "UNIQUE", "ASCENDING")
arcpy.conversion.FeatureClassToFeatureClass(filegdb+r"\PlatsInParcels", filegdb, "Parcels_ID", '', r'JOINPIN "JOINPIN" true true false 23 Text 0 0,First,#,PlatsInParcels,JOINPIN,0,23', '')
arcpy.management.AddJoin("Parcels_ID", "JOINPIN", filegdb+r"\ParcelZoning", "JOINPIN", "KEEP_ALL")
arcpy.management.FeatureToPoint("LU2020", filegdb+r"\LandUsePoints", "INSIDE")
arcpy.analysis.SpatialJoin("Parcels_ID", filegdb+r"\LandUsePoints", filegdb+r"\ParcelZoningLandUse", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField(filegdb+r"ParcelZoningLandUse", "Join_Count", "LandUse_Count", "LandUses Per Parcel", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.FeatureToPoint("LU2020", filegdb+r"\LandUsePoints", "INSIDE")
arcpy.analysis.SpatialJoin("Parcels_ID", filegdb+r"\LandUsePoints", filegdb+r"\ParcelZoningLandUse", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField(filegdb+r"\ParcelZoningLandUse", "Join_Count", "LandUse_Count", "LandUses Per Parcel", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.analysis.SummarizeWithin("ParcelZoningLandUse", "BuildingFootprints", filegdb+r"\ParcelZoningLandUseBuildings", "KEEP_ALL", "Shape_Area Sum", "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.management.AlterField(filegdb+r"\ParcelZoningLandUseBuildings", "SUM_Area_SQUAREFEET", "BuildingFootprintArea", "Building Footprint Square Footage", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.conversion.FeatureClassToFeatureClass("Building Permit", filegdb, "NewPermitsResidential", "Issued >= timestamp '2018-01-01 00:00:00' And Classification IN ('COMMERCIAL NEW MULTI-FAMILY', 'RESIDENTIAL NEW DUPLEX', 'RESIDENTIAL NEW SINGLE FAMILY')", '#', '')
arcpy.analysis.SpatialJoin("ParcelZoningLandUseBuildings", "NewPermitsResidential", filegdb+r"\ParcelZoningLandUseBuildingsPermits", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", None, '')
arcpy.management.Dissolve("ParcelZoningLandUseBuildingsPermits", filegdb+r"\Subdivisions", "SUBDIVISION;NAME;Zoning", "Parcel_zone_Count SUM;Join_Count SUM;Lots_in_Parcel SUM;flivunits SUM;SUM_Area_SQUAREFEET SUM;Permit COUNT", "SINGLE_PART", "DISSOLVE_LINES")

<Result 'C:\\Users\\kgonterwitz\\Documents\\ArcGIS\\Projects\\Kyle Project 2\\BuildingSum20210401.gdb\\ParcelZoningLandUseBuildingsPermits'>

that seems to be working. 

This is for testing.  I am expanding the range of recent permits to include all issued after 1/1/2018.

I want to tweak this code to see if I can get date ranges on permit and yearbuilt histories.


In [ ]:
filegdb = r"C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210401.gdb"
arcpy.conversion.FeatureClassToFeatureClass("Plats", filegdb, "PlatsCopy", "#", '#', '')
arcpy.management.AddField(filegdb+r"\PlatsCopy", "LotBlock", "TEXT", None, None, 24, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(filegdb+r"\PlatsCopy", "LotBlock", 'str(!BLOCK!)+"-"+ str(!LOT!)', "PYTHON3", '', "TEXT")
arcpy.management.FeatureToPoint("PlatsCopy", filegdb+r"\PlatsCopyPoints", "INSIDE")
arcpy.analysis.SpatialJoin("Parcels", "PlatsCopyPoints", filegdb+r"\PlatsInParcels", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", None, '')
arcpy.management.AlterField("PlatsInParcels", "Join_Count", "Lots_in_Parcel", "Platted Lots Per Parcel", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.FeatureToPoint("PlatsInParcels", filegdb+r"\PlatsInParcelPoint", "INSIDE")
arcpy.analysis.SpatialJoin(filegdb+r"\PlatsInParcelPoint", "Lawrence Zoning", filegdb+r"\ParcelZoning", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField("ParcelZoning", "Join_Count", "Parcel_zone_Count", "Zonings Per Parcel", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.AddIndex(filegdb+r"\PlatsInParcels", "JOINPIN", "JOINPIN", "UNIQUE", "ASCENDING")
arcpy.management.AddIndex(filegdb+r"\ParcelZoning", "JOINPIN", "JOINPINz", "UNIQUE", "ASCENDING")
arcpy.conversion.FeatureClassToFeatureClass(filegdb+r"\PlatsInParcels", filegdb, "Parcels_ID", '', r'JOINPIN "JOINPIN" true true false 23 Text 0 0,First,#,PlatsInParcels,JOINPIN,0,23', '')
arcpy.management.AddJoin("Parcels_ID", "JOINPIN", filegdb+r"\ParcelZoning", "JOINPIN", "KEEP_ALL")
arcpy.management.FeatureToPoint("LU2020", filegdb+r"\LandUsePoints", "INSIDE")
arcpy.analysis.SpatialJoin("Parcels_ID", filegdb+r"\LandUsePoints", filegdb+r"\ParcelZoningLandUse", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField(filegdb+r"ParcelZoningLandUse", "Join_Count", "LandUse_Count", "LandUses Per Parcel", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.FeatureToPoint("LU2020", filegdb+r"\LandUsePoints", "INSIDE")
arcpy.analysis.SpatialJoin("Parcels_ID", filegdb+r"\LandUsePoints", filegdb+r"\ParcelZoningLandUse", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField(filegdb+r"\ParcelZoningLandUse", "Join_Count", "LandUse_Count", "LandUses Per Parcel", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.analysis.SummarizeWithin("ParcelZoningLandUse", "BuildingFootprints", filegdb+r"\ParcelZoningLandUseBuildings", "KEEP_ALL", "Shape_Area Sum", "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.management.AlterField(filegdb+r"\ParcelZoningLandUseBuildings", "SUM_Area_SQUAREFEET", "BuildingFootprintArea", "Building Footprint Square Footage", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")
arcpy.conversion.FeatureClassToFeatureClass("Building Permit", filegdb, "NewPermitsResidential", "Issued >= timestamp '2018-01-01 00:00:00' And Classification IN ('COMMERCIAL NEW MULTI-FAMILY', 'RESIDENTIAL NEW DUPLEX', 'RESIDENTIAL NEW SINGLE FAMILY')", '#', '')
arcpy.analysis.SpatialJoin("ParcelZoningLandUseBuildings", "NewPermitsResidential", filegdb+r"\ParcelZoningLandUseBuildingsPermits", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", None, '')
arcpy.management.Dissolve("ParcelZoningLandUseBuildingsPermits", filegdb+r"\Subdivisions", "SUBDIVISION;NAME;Zoning", "Parcel_zone_Count SUM;Join_Count SUM;Lots_in_Parcel SUM;flivunits SUM;SUM_Area_SQUAREFEET SUM;Permit COUNT", "SINGLE_PART", "DISSOLVE_LINES")